![Pandas Logo](./images/pandas_logo.png)

- [Estructuras de datos](#estructuras)
    - [Series](#series)
    - [DataFrame](#dataframe)
    - [Objetos Índice](#indices)
- [Funcionalidad básica](#funciones)
    - [Reindexación](#reindexacion)
    - [Eliminando entradas de un eje](#eliminar_eje)
    - [Indexación, Selección y filtrado](#indexacion)
    - [Aritmetica y Alineación de datos](#aritmetica)
    - [Aplicación y mapeo de funciones](#mapeo)
    - [Ordenación y clasificación](#ordenacion)
    - [Índices sobre ejes con etiquetas duplicadas](#etiquetas_duplicadas)
    - [Datos duplicados](#datos_duplicados)
- [Usando estadísticas descriptivas](#estadisticas)
    - [Correlación y Covarianza](#correlacion)
    - [Valores únicos, recuentos de valor y membresía](#recuentos)

---

# Introducción a Pandas

`Pandas` es una herramienta esencial para el análisis de datos. Contiene estructuras de datos y herramientas de manipulación de datos diseñadas para que la limpieza y el análisis de datos sea rápido y fácil en Python. Pandas se usa a menudo junto con herramientas de computación numérica como `NumPy` y `SciPy`, bibliotecas analíticas como `statsmodels` y `scikit-learn`, y bibliotecas de visualización de datos como `matplotlib`. Pandas adopta partes significativas del estilo idiomático de NumPy como la computación basada en matrices, especialmente las funciones basadas en matrices y una preferencia por el procesamiento de datos sin bucles.

Si bien en Pandas se adoptan muchos elementos de codificación de NumPy, la mayor diferencia es que Pandas está diseñado para trabajar con datos tabulares o heterogéneos, mientra que NumPy, por contraste, es más adecuado para trabajar con datos de matrices numéricas homogéneas.

In [1]:
import pandas as pd
import numpy as np
pd.__version__

'2.2.3'

<a id="estructuras"></a>
## Estructuras de datos

Las dos estructuras de datos principales de Pandas son las series (`Series`) y los marcos de datos (`DataFrame`). Si bien no son una solución universal para todos los problemas, proporcionan una base sólida y fácil de usar para la mayoría de las aplicaciones.

<a id="series"></a>
### Series
Una serie es un objeto similar a una matriz unidimensional que contiene una secuencia de valores (de tipos similares a los tipos NumPy) y una matriz asociada de etiquetas de datos, denominada índice. La serie más simple está formada a partir de una matriz de datos. Si no especificamos un índice para los datos, se crea uno predeterminado que consiste en números enteros de 0 a N - 1 (donde N es la longitud de los datos):

In [2]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Se pueden obtener los valores y el índice de la serie a través de sus atributos `values` e `index` respectivamente:

In [3]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [4]:
obj.index  # como range(4)

RangeIndex(start=0, stop=4, step=1)

Es posible proporcionar el índice a utilizar cuando se crea una serie:

In [5]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [6]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

A diferencia de los arrays en Numpy, se pueden utilizar las etiquetas de los índices para acceder a valores o conjuntos de valores:

In [7]:
obj2['a']

-5

In [8]:
obj2['d'] = 6
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [9]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

Aquí `['c', 'a', 'd']` se interpreta como una lista de índices, aunque contenga cadenas en lugar de números enteros.

El uso de funciones NumPy u operaciones similares a NumPy, como el filtrado con una matriz booleana, la multiplicación escalar o la aplicación de funciones matemáticas, siempre mantiene el valor del índice:

In [10]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [11]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

Si los datos que se proporcionan para crear un serie es un valor escalar, se debe proporcionar un índice. El valor se repetirá para que coincida con la longitud del índice.

In [12]:
pd.Series(5., index=['a', 'b', 'c', 'd', 'e'])

a    5.0
b    5.0
c    5.0
d    5.0
e    5.0
dtype: float64

Cuando se trabaja con matrices NumPy sin procesar, normalmente no es necesario realizar un bucle de valor por valor. Lo mismo ocurre cuando se trabaja con Series en pandas. Las series también se pueden pasar a la mayoría de los métodos NumPy que esperan un ndarray.

In [13]:
from numpy import exp
exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Una diferencia clave entre Series y ndarray es que las operaciones entre Series alinean automáticamente los datos en función de la etiqueta. Por lo tanto, se pueden escribir cálculos sin tener en cuenta si las Series involucradas tienen las mismas etiquetas. 

In [14]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [15]:
obj3 = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
obj3

a    1
b    2
c    3
d    4
dtype: int64

In [16]:
obj2 + obj3

a    -4
b     9
c     6
d    10
dtype: int64

El resultado de una operación entre Series no alineadas tendrá la unión de los índices involucrados. Si no se encuentra una etiqueta en una Serie u otra, el resultado se marcará como falta `NaN`. Ser capaz de escribir código sin hacer una alineación de datos explícita otorga inmensa libertad y flexibilidad en el análisis e investigación de datos interactivos. Las características integradas de alineación de datos de las estructuras de datos de pandas diferencian a los pandas de la mayoría de las herramientas relacionadas para trabajar con datos etiquetados.

In [17]:
obj4 = pd.Series([1, 1, 1, 1], index=['x', 'y', 'c', 'd'])
obj4

x    1
y    1
c    1
d    1
dtype: int64

In [18]:
obj2 + obj4

a    NaN
b    NaN
c    4.0
d    7.0
x    NaN
y    NaN
dtype: float64

Otra forma de pensar en una serie es como un diccionario ordenado de longitud fija, ya que es un mapeo de valores de índice a valores de datos. Se puede utilizar en muchos contextos en los que puede usar un diccionario (`dict`):

In [19]:
'b' in obj2

True

In [20]:
'e' in obj2

False

Podemos crear series a partir de diccionarios Python:

In [21]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

Cuando se pasa un diccionario, el índice en la serie resultante tendrá las claves del diccionario ordenadas. Se puede anular el orden pasando las claves del diccionario en el orden en que desea que aparezcan en la serie resultante:

In [22]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

En el ejemplo anterior se encuentras tres valores en `sdata` que se ubican en las posiciones apropiadas, pero como no se encontró ningún valor para 'California', aparece como `NaN` (*Not an Number*) que se utiliza en pandas para marcar valores faltantes o `NA` (*Not Available*). Dado que 'Utah' no se incluyó en los estados, se excluye del objeto resultante.

>En pandas, se adopta la convención utilizada en el lenguaje de programación `R` al referirnos a datos faltantes como `NA`, que significa no disponible. En aplicaciones de estadísticas, los datos de NA pueden ser datos que no existen o que existen pero que no se observaron.

Las funciones y/o métodos `isnull` y `notnull` en pandas se usan para detectar datos faltantes:

In [23]:
pd.isnull(obj4) # equivalente a obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [24]:
pd.notnull(obj4) # equivalente a obj4.notnull()

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

Una característica muy útil de las series para muchas aplicaciones es que se alinéan automáticamente por índice en las operaciones aritméticas, de forma similar a las operaciones de unión de las bases de datos relacionales:

In [25]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [26]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [27]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Tanto las series como su índice tienen un atributo de nombre (`name`), que se integra con otras áreas clave de la funcionalidad de los pandas:

In [28]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

El índice de una serie se puede modificar *in situ* por asignación:

In [29]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

<a id="dataframe"></a>
### DataFrame

Un marco de datos o `DataFrame` representa una tabla de datos rectangular y contiene una colección ordenada de columnas, cada una de las cuales puede ser un tipo de valor diferente (numérico, de cadena, booleano, etc.). El DataFrame tiene un índice de fila y columna. Se puede considerar como un diccionario de series que comparten el mismo índice. 

Internamente los datos se almacenan como uno o más bloques bidimensionales en lugar de una lista, diccionario, o alguna otra colección de arrays unidimensionales. 

> Mientras que un `DataFrame` es físicamente bidimensional, se puede usar para representar datos de dimensiones más altas en un formato tabular utilizando la indexación jerárquica.

Hay muchas formas de construir un `DataFrame`, aunque una de las más comunes es desde un diccionario de listas de igual longitud o desde matrices NumPy:

In [30]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

El DataFrame resultante tendrá su índice asignado automáticamente como ocurre con las series, y las columnas se colocan de forma ordenada:

In [31]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [32]:
# El método head permite mostrar las 5 primeras filas por defecto
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [33]:
# El método tail permite mostrar las 5 últimas filas por defecto
frame.tail(3)

,state,year,pop
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


Si especifica una secuencia de columnas, el `DataFrame` mostrará sus columnas en la secuencia indicada:

In [34]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


Si se indica una columna que no está contenida en el diccionario, aparecerá con valores `NA` (`NaN`) en el resultado:

In [35]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four', 'five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [36]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

Una columna en un `DataFrame` se puede recuperar como una `Serie`, ya sea con notación tipo diccionario o vía atributo:

In [37]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [38]:
# sólo si el nombre de la columna no coincide con una propiedad o método del DataFrame
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

Las series devueltas tienen el mismo índice que el DataFrame, y su atributo `name` se ha establecido correctamente.


Las filas también se pueden recuperar por posición o nombre con el atributo especial `loc`:

In [39]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

Las columnas pueden ser modificadas por asignación:

In [40]:
# asignación de un valor escalar
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [41]:
# asignación de un vector
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


Cuando se están asignando listas o matrices a una columna, la longitud de valores debe coincidir con la longitud del DataFrame. Si se asigna una serie, sus etiquetas se realinearán exactamente al índice del DataFrame, insertando valores `NA` en el resto de huecos:

In [42]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


Asignar una columna que no existe creará una nueva columna. La palabra clave `del` borrará columnas como con un diccionario:

In [43]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [44]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

La columna devuelta de la indexación de un DataFrame es una vista de los datos subyacentes, no una copia. Por lo tanto, cualquier modificación *in situ* de la serie se reflejará en el DataFrame. La columna se puede copiar explícitamente con el método `copy` de la serie.

Otra forma común de crear un DataFrame es utilizar diccionarios anidados. Pandas interpretará las claves del diccionario externo como las columnas y las claves de diccionario interno como los índices de fila:

In [45]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


Se puede transponer el DataFrame (intercambiar filas y columnas) con una sintaxis similar a una matriz NumPy:

In [46]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


Las claves en los diccionarios internos se combinan y ordenan para formar el índice en el resultado. Esto no es cierto si se especifica un índice explícito:

In [47]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


Los diccionarios de series se tratan de la misma manera:

In [48]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


Si el atributo `name` del índice y de las columnas de un DataFrame están fijados, sus valores serán mostrados:

In [49]:
frame3.index.name = 'year'
frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


Al igual que con las series, el atributo `values` devuelve los datos contenidos en el DataFrame como un ndarray bidimensional:

In [50]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

Si las columnas del DataFrame son de tipos diferentes, el tipo de la matriz de valores se elegirá para acomodar todas las columnas:

In [51]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

La siguiente tabla muestra las posibles entradas para la construcción de un `DataFrame`:

|Tipo|Descripción|
|---|---|
|2D ndarray|Una matriz de datos, pasando las etiquetas opcionales de fila y columna|
|dict de matrices, lists, o tuples|Cada secuencia se convierte en una columna en el DataFrame; Todas las secuencias deben tener la misma longitud|
|NumPy structured/record array|Se tratan como el caso del "diccionario de matrices"|
|dict de Series|Cada valor se convierte en una columna; los índices de cada Serie se unen para formar el índice de la fila del resultado si no se pasa un índice explícito|
|dict de dicts|Cada diccionario interno se convierte en una columna; Las claves están unidas para formar el índice de fila como en el caso "dict de Series"|
|List de dicts o Series|Cada elemento se convierte en una fila en el marco de datos; La unión de las claves de dict o los índices de serie se convierten en las etiquetas de columna de DataFrame|
|List de lists o tuples|Tratada como el caso "2D ndarray"|
|Another DataFrame|Los índices de DataFrame se utilizan a menos que se pasen diferentes|
|NumPy MaskedArray|Al igual que en el caso de "ndarray 2D", los valores enmascarados se convierten en NA/faltantes en el resultado del marco de datos|


<a id="indices"></a>
### Objetos Índice

Los objetos índice (`Index`) de pandas son responsables de mantener las etiquetas del eje y otros metadatos (como el nombre o los nombres del eje). Cualquier matriz u otra secuencia de etiquetas que utilice al construir una serie o un DataFrame se convierte internamente en un `Index`:

In [52]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

Los índices son inmutables:

In [53]:
index[1:]

Index(['b', 'c'], dtype='object')

In [54]:
index[1] = 'd'  # TypeError

TypeError: Index does not support mutable operations

La inmutabilidad hace que sea seguro compartir objetos de índice entre estructuras de datos:

In [55]:
labels = pd.Index(np.arange(3))
labels

Index([0, 1, 2], dtype='int32')

In [56]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [57]:
obj2.index is labels

True

Además de ser similar a una matriz, un índice también se comporta como un conjunto de tamaño fijo:

In [58]:
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [59]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [60]:
'Ohio' in frame3.columns

True

In [61]:
2003 in frame3.index

False

A diferencia de Python, los índices de pandas pueden contaner valores duplicados. Las selecciones con etiquetas duplicadas seleccionarán todas las apariciones de esa etiqueta.

In [62]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

Los índices pueden tener múltiples niveles:

In [63]:
arrays = [[0, 0, 1, 1, 2, 2],
          ['one', 'two', 'one', 'two', 'one', 'two']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
index

MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two'),
            (2, 'one'),
            (2, 'two')],
           names=['first', 'second'])

In [64]:
index = pd.MultiIndex.from_product([range(3), ['one', 'two']], names=['first', 'second'])
index

MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two'),
            (2, 'one'),
            (2, 'two')],
           names=['first', 'second'])

In [65]:
pd.Series(np.random.randn(6), index=index)

first  second
0      one       0.190888
       two      -0.507893
1      one      -0.808208
       two       1.738928
2      one       1.522476
       two       0.442412
dtype: float64

In [66]:
index.levels[1]

Index(['one', 'two'], dtype='object', name='second')

In [67]:
index.names

FrozenList(['first', 'second'])

In [68]:
index.set_levels(["a", "b"], level=1)

MultiIndex([(0, 'a'),
            (0, 'b'),
            (1, 'a'),
            (1, 'b'),
            (2, 'a'),
            (2, 'b')],
           names=['first', 'second'])

In [69]:
index.get_level_values(0)

Index([0, 0, 1, 1, 2, 2], dtype='int64', name='first')

>**Importante**: Aunque un índice puede contener valores perdidos (`NaN`), debe evitarse si no se desea ningún resultado inesperado. Por ejemplo, algunas operaciones excluyen implícitamente los valores perdidos.

Los índices tiene una serie de métodos y propiedades para establecer la lógica y obtener informaciones comunes sobre los datos que contienen, los más útiles se resumen en la siguiente tabla:

|Método|Descripción|
|---|:---|
|append|Concatena los índices produciendo un nuevo índice|
|difference|Devuelve la diferencia de conjuntos como un índice|
|intersection|Devuelve la intersección de conjuntos como un índice|
|union|Devuelve el conjunto unión como un índice|
|isin|Define una matriz booleana que indica si cada valor está contenido en la colección pasada|
|delete|Define un nuevo índice eliminando el elemento `i` |
|drop|Define un nuevo índice eliminando los elementos indicados|
|insert|Define un nuevo índice insertando los elementos indicados|
|is_monotonic|Devuelve `True` si cada elemento es mayor o igual que el elemento anterior|
|is_unique|Devuele `True` si el Índice no tiene valores duplicados|
|unique|Define un nuevo índice sin elementos duplicados|


<a id="funciones"></a>
## Funcionalidad básica

Esta sección incluye los mecanismos fundamentales de la interacción con los datos contenidos en una serie (`Serie`) o un marco de datos (`DataFrame`).

<a id="reindexacion"></a>
### Reindexación

El método `set_index` que toma un nombre de columna (para un índice regular) o una lista de nombres de columna (para un `MultiIndex`). Para crear un nuevo DataFrame re-indexado:

In [70]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2],
        'info': [5, 7, 6, 4, 9, 2]}
frame = pd.DataFrame(data)
frame

,state,year,pop,info
0,Ohio,2000,1.5,5
1,Ohio,2001,1.7,7
2,Ohio,2002,3.6,6
3,Nevada,2001,2.4,4
4,Nevada,2002,2.9,9
5,Nevada,2003,3.2,2


In [71]:
indexed1 = frame.set_index('state')
indexed1

,year,pop,info
state,,,
Ohio,2000,1.5,5
Ohio,2001,1.7,7
Ohio,2002,3.6,6
Nevada,2001,2.4,4
Nevada,2002,2.9,9
Nevada,2003,3.2,2


In [72]:
indexed2 = frame.set_index(['state', 'year'])
indexed2

pop  info
state  year           
Ohio   2000  1.5     5
       2001  1.7     7
       2002  3.6     6
Nevada 2001  2.4     4
       2002  2.9     9
       2003  3.2     2

In [73]:
indexed3 = frame.set_index('state', drop=False)
indexed3

,state,year,pop,info
state,,,,
Ohio,Ohio,2000,1.5,5
Ohio,Ohio,2001,1.7,7
Ohio,Ohio,2002,3.6,6
Nevada,Nevada,2001,2.4,4
Nevada,Nevada,2002,2.9,9
Nevada,Nevada,2003,3.2,2


In [74]:
indexed3.set_index('year', append=True, inplace=True)
indexed3

state  pop  info
state  year                   
Ohio   2000    Ohio  1.5     5
       2001    Ohio  1.7     7
       2002    Ohio  3.6     6
Nevada 2001  Nevada  2.4     4
       2002  Nevada  2.9     9
       2003  Nevada  3.2     2

Hay una nueva función en DataFrame llamada `reset_index` que transfiere los valores del índice a las columnas del DataFrame y establece un índice entero simple. Esta es la operación inversa de `set_index`.

In [75]:
indexed2.reset_index()

,state,year,pop,info
0,Ohio,2000,1.5,5
1,Ohio,2001,1.7,7
2,Ohio,2002,3.6,6
3,Nevada,2001,2.4,4
4,Nevada,2002,2.9,9
5,Nevada,2003,3.2,2


In [76]:
indexed2.reset_index(level=1)

,year,pop,info
state,,,
Ohio,2000,1.5,5
Ohio,2001,1.7,7
Ohio,2002,3.6,6
Nevada,2001,2.4,4
Nevada,2002,2.9,9
Nevada,2003,3.2,2


Otro método importante en pandas es `reindex`que permite crear un nuevo objeto con los datos ajustados a un nuevo índice. Cuando reindexamos una serie, se reorganizan los datos de acuerdo con el nuevo índice, introduciendo valores `NaN` para los ínices no presentes:

In [77]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [78]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

Para datos ordenados como series de tiempo, puede ser conveniente hacer una interpolación o llenado de valores al reindexar. La opción `method` nos permite hacer esto, utilizando un método como `ffill`, que rellena los valores hacia adelante:

In [79]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [80]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

Con DataFrame, la reindexación puede alterar el índice (fila), las columnas o ambos. Cuando se pasa solo una secuencia, se reindexan las filas en el resultado:

In [81]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [82]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


Las columnas pueden ser reindexadas con la palabra clave `columns`:

In [83]:
states = ['Texas', 'Utah', 'California']
frame2.reindex(columns=states)

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


La siguiente tabla muestra diferentes argumentos de la función de reindexación:

|Argumento|Descripción|
|---|---|
|index|Nueva secuencia para usar como índice. Puede ser una instancia de índice o cualquier otra estructura de datos de Python similar a una secuencia. Un índice se utilizará exactamente como está sin ninguna copia|
|method|Método de interpolación (relleno): `ffill` se llena hacia adelante, mientras que `bfill` se llena hacia atrás|
|fill_value|Valor de reemplazo que se utilizará cuando se introducen datos faltantes mediante la reindexación|
|limit|Cuando se interpola hacia adelante o hacia atrás, el espacio máximo se puede llenar (en número de elementos)|
|tolerance|Cuando se interpola hacia adelante o hacia atrás, la separación máxima de tamaño (en distancia numérica absoluta) se debe completar para coincidencias inexactas|
|level|Ajusta un índice simple con el nivel de MultiIndex; en otro caso se selecciona un subconjunto del mismo|
|copy|Si es `True`, siempre se copian los datos subyacentes, incluso si el nuevo índice es equivalente al índice anterior; si es `False`, no copia los datos cuando los índices son equivalentes|


<a id="eliminar_eje"></a>
### Eliminando entradas de un eje

Eliminar una o más entradas de un eje es fácil si ya se tiene una matriz o lista de índices sin esas entradas. El método `drop` devuelve un nuevo objeto con el valor o valores eliminados del eje indicado:

In [84]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [85]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [86]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

En un DataFrame, los valores de índice se pueden eliminar de cualquiera de los ejes.

In [87]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Si se llama a `drop` con una secuencia de etiquetas, se eliminarán los valores de las etiquetas de las filas (eje 0):

In [88]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


Se pueden eliminar valores de las columnas pasando `axis = 1` o `axis = 'columns'`:

In [89]:
data.drop('two', axis=1)
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


Muchas funciones, como `drop`, que modifican el tamaño o la forma de una serie o marco de datos, pueden manipular un objeto in situ sin devolver un nuevo objeto:. Hay que tener cuidado con `inplace`, ya que destruye los datos que se eliminan.

In [90]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

<a id="indexacion"></a>
### Indexación, Selección y filtrado

La información de etiquetado del eje en los objetos pandas sirve para muchos propósitos:

- Identifica datos (es decir, proporciona metadatos) utilizando indicadores conocidos, importantes para el análisis, la visualización y la visualización de la consola interactiva.
- Permite la alineación automática y explícita de los datos.
- Permite obtener y configurar intuitivamente los subconjuntos del conjunto de datos.
     
La indexación de series (`obj [...]`) funciona de manera análoga a la indexación de matrices en NumPy, excepto que pueden usar los valores de índice de la serie en lugar de solo números enteros. También actúa como un diccionario estándar de Python. Si tenemos en cuenta estas dos analogías, nos ayudará a comprender los patrones de indexación y selección de datos en estas matrices.

In [91]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

La selección de objetos ha tenido varias adiciones solicitadas por el usuario para admitir una indexación más explícita basada en la ubicación. Pandas ahora admite diferentes tipos de indexación de ejes múltiples.

- Una sola etiqueta, por ej. 5 o 'a' (Tenga en cuenta que 5 se interpreta como una etiqueta del índice. Este uso no es una posición entera a lo largo del índice).
- Una lista o conjunto de etiquetas ['a', 'b', 'c'], [1, 2, 3].
- Un objeto de división con etiquetas 'a' : 'f' (Tenga en cuenta que, a diferencia de las secciones típicas de Python, se incluyen tanto el inicio como el final, cuando están presentes en el índice) o '1 : 3'
- Una matriz booleana
- Una función que se puede llamar con un argumento (la serie que llama o el marco de datos) y que devuelve un resultado válido para la indexación (uno de los anteriores).

In [92]:
# Una sola etiqueta
obj['b']

1.0

In [93]:
# Una sola etiqueta
obj[1]

C:\Users\mfcardenas\AppData\Local\Temp\ipykernel_38348\102297531.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[1]


1.0

In [94]:
# Un objeto de división con etiquetas
# el filtrado con etiquetas se comporta de manera diferente al de Python en que el punto final es inclusivo
obj['a':'c']

a    0.0
b    1.0
c    2.0
dtype: float64

In [95]:
# Un objeto de división con enteros 
obj[0:3]

a    0.0
b    1.0
c    2.0
dtype: float64

In [96]:
# Un objeto de división con enteros
obj[::-1]

d    3.0
c    2.0
b    1.0
a    0.0
dtype: float64

In [97]:
# Una lista o conjunto de etiquetas
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [98]:
# Una lista o conjunto de enteros
obj[[1, 0, 3]]

C:\Users\mfcardenas\AppData\Local\Temp\ipykernel_38348\2799166618.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[[1, 0, 3]]


b    1.0
a    0.0
d    3.0
dtype: float64

In [99]:
# Una matriz booleana
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [100]:
# Una función
obj[lambda s: s < 2]

a    0.0
b    1.0
dtype: float64

La indexación en un DataFrame permite recuperar una o más columnas con un solo valor o secuencia:

In [101]:
import numpy as np
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [102]:
# Una sola etiqueta
# data.two
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [103]:
# Un conjunto de etiquetas
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


Esta indexación tiene algunos casos especiales. Primero, al pasar un solo elemento o una lista al operador `[]`, selecciona las columnas.

In [104]:
# Un objeto de división con enteros
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [105]:
# Una matriz booleana
data[data < 5]

,one,two,three,four
Ohio,0.0,1.0,2.0,3.0
Colorado,4.0,NaN,NaN,NaN
Utah,NaN,NaN,NaN,NaN
New York,NaN,NaN,NaN,NaN


La selección de valores de una serie con un vector booleano generalmente devuelve un subconjunto de los datos. Para garantizar que la salida de selección tenga la misma forma que los datos originales, puede usar el método `where` en Series y DataFrame.

Para devolver solo las filas seleccionadas:

In [106]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Para devolver una serie de la misma forma que el original:

In [107]:
data.where(data['three'] > 5)

,one,two,three,four
Ohio,NaN,NaN,NaN,NaN
Colorado,4.0,5.0,6.0,7.0
Utah,8.0,9.0,10.0,11.0
New York,12.0,13.0,14.0,15.0


Además, `where` toma un otro argumento opcional para la sustitución de valores donde la condición es `False`, en la copia devuelta.

In [108]:
data.where(data['three'] > 5, -1)

,one,two,three,four
Ohio,-1,-1,-1,-1
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Es posible que desee establecer valores basados en algunos criterios booleanos. Esto se puede hacer de manera intuitiva así:

In [109]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Por defecto, `where` devuelve una copia modificada de los datos. Hay un parámetro opcional `inplace` los datos originales puedan modificarse sin crear una copia:

In [110]:
data.where(data > 5, -1, inplace=True)
data

,one,two,three,four
Ohio,-1,-1,-1,-1
Colorado,-1,-1,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selección con `loc` e `iloc`
Estas convenciones de segmentación e indización pueden ser una fuente de confusión. Por ejemplo, si la serie tiene un índice entero explícito, una operación de indexación como `data[1]` utilizará los índices explícitos, mientras que una operación de segmentación como `data[1 : 3]` usará el índice implícito del estilo de Python.

In [111]:
other_data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
other_data

1    a
3    b
5    c
dtype: object

In [112]:
# índice explícito 
other_data[1]

'a'

In [113]:
# índice implícito
other_data[1:3]

3    b
5    c
dtype: object

Debido a esta posible confusión en el caso de los índices enteros, Pandas proporciona algunos atributos de indexación especiales. Primero, el atributo `loc` permite indexar y segmentar con referencia al índice explícito.

**`.loc`** se basa principalmente en etiquetas, pero también se puede utilizar con una matriz booleana. `.loc` generará `KeyError` cuando no se encuentren los elementos. Las entradas permitidas son:

In [114]:
other_data.loc[1]

'a'

In [115]:
other_data.loc[1:3]

1    a
3    b
dtype: object

In [116]:
data.loc[lambda df: df.two > 5, :]

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


El atributo `iloc` permite indexar y segmentar con referencia al índice implícito del estilo de Python.

**`.iloc`** se basa principalmente en la posición de enteros (de 0 a longitud-1 del eje), pero también se puede usar con una matriz booleana. `.iloc` generará `IndexError` si un indizador solicitado está fuera de los límites, excepto los indizadores de segmentos que permiten la indexación fuera de los límites. (Esto se ajusta a la semántica de segmentos de Python/NumPy).

In [117]:
other_data.iloc[1]

'b'

In [118]:
other_data.iloc[1:3]

3    b
5    c
dtype: object

In [119]:
# Selecciona todas las filas y dos columnas
data.iloc[:, [0, 1]]

,one,two
Ohio,-1,-1
Colorado,-1,-1
Utah,8,9
New York,12,13


Adicionalmente `loc` permite la utilización de etiquetas, mientras que `iloc`sólo permite enteros:

In [120]:
# Selecciona una fila y varias columnas
data.loc['Colorado', ['two', 'three']]

two     -1
three    6
Name: Colorado, dtype: int32

In [121]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [122]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,-1,-1
Utah,11,8,9


Ambas funciones de indexación funcionan con segmentos, además de etiquetas individuales o listas de etiquetas:

In [123]:
data.loc[:'Utah', 'two']

Ohio       -1
Colorado   -1
Utah        9
Name: two, dtype: int32

In [124]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,-1,-1,6
Utah,8,9,10
New York,12,13,14


In [125]:
data.one.loc[lambda s: s > 4]

Utah         8
New York    12
Name: one, dtype: int32

Dado que la indexación con `[]` debe manejar muchos casos (acceso de etiqueta única, segmentación, indexación booleana, etc.), tiene un poco de sobrecarga para poder averiguar lo que está pidiendo. Si solo desea acceder a un valor escalar, la forma más rápida es utilizar los métodos `at` e `iat`, que se implementan en todas las estructuras de datos.

De forma similar a `loc`, `at` proporciona búsquedas escalares basadas en etiquetas, mientras que `iat` proporciona búsquedas basadas en enteros de manera análoga a `iloc`.

In [126]:
data.at['Utah', 'two']

9

In [127]:
data.iat[2, 1]

9

Por último, los objetos DataFrame tienen un método `query` que permite la selección usando una expresión.

In [128]:
# equivalente a data[data.one<data.two & data.two<data.three]
data.query('one<two & two<three')

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [129]:
# equivalente a data[data.one.isin(data.two)]
data.query('one in two')

,one,two,three,four
Ohio,-1,-1,-1,-1
Colorado,-1,-1,6,7


>**Nota**: DataFrame `query` usando expresiones numéricas es ligeramente más rápido que Python para marcos de datos grandes.

La siguiente tabla muestra algunas de los modos de indexación de un DataFrame:

|Tipo|Descripción|
|---|---|
|df[val]|Selecciona una sola columna o secuencia de columnas del DataFrame|
|df.loc[val]|Selecciona una sola fila o un subconjunto de filas del DataFrame por etiqueta|
|df.loc[:, val]|Selecciona una sola columna o subconjunto de columnas por etiqueta|
|df.loc[val1, val2]|Seleccione ambas filas y columnas por etiqueta|
|df.iloc[where]|Selecciona una sola fila o un subconjunto de filas del marco de datos por posición entera|
|df.iloc[:, where]|Selecciona una sola columna o un subconjunto de columnas por posición entera|
|df.iloc[where_i, where_j]|Selecciona tanto filas como columnas por posición entera|
|df.at[label_i, label_j]|Seleccione un solo valor escalar por fila y columna etiquetada|
|df.iat[i, j]|Seleccione un único valor escalar por fila y posición de columna (enteros) |
|reindex method|Seleccione filas o columnas etiquetadas|
|get_value, set_value methods|Seleccione un solo valor por fila y columna etiquetada|


#### Indexación avanzada con índice jerárquico
La integración sintáctica de `MultiIndex` en la indexación avanzada con `.loc` es complicada. En general, las claves `MultiIndex` toman la forma de tuplas. Por ejemplo, lo siguiente funciona como cabría esperar:

In [130]:
arrays = [np.array(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux']),
          np.array(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'])]
df = pd.DataFrame(np.random.randn(8, 3), index=arrays, columns=['A', 'B', 'C'])
df

A         B         C
bar one  0.689453 -1.036817 -0.299809
    two  1.383091 -0.726800 -0.190741
baz one -0.078887  0.328318  0.892238
    two -0.085608  0.035008 -0.978340
foo one  0.624355  0.751345 -0.519235
    two  0.387675  0.630915  1.287880
qux one  1.758450  0.880635 -0.436806
    two -0.408464 -1.981943  0.894582

In [131]:
# df.loc ['bar', 'two'] también funcionaría, pero esta notación puede llevar a la ambigüedad en general.
df.loc[('bar', 'two')]

A    1.383091
B   -0.726800
C   -0.190741
Name: (bar, two), dtype: float64

Si se desea indexar una columna específica con `.loc`, se debe usar una tupla como:

In [132]:
df.loc[('bar', 'two'), 'A']

1.3830906760144743

In [133]:
# equivalente a df.loc[('bar',),]
df.loc['bar']

,A,B,C
one,0.689453,-1.036817,-0.299809
two,1.383091,-0.726800,-0.190741


In [134]:
df.loc['baz':'foo']

A         B         C
baz one -0.078887  0.328318  0.892238
    two -0.085608  0.035008 -0.978340
foo one  0.624355  0.751345 -0.519235
    two  0.387675  0.630915  1.287880

In [135]:
df.loc[('baz', 'two'):('qux', 'one')]

A         B         C
baz two -0.085608  0.035008 -0.978340
foo one  0.624355  0.751345 -0.519235
    two  0.387675  0.630915  1.287880
qux one  1.758450  0.880635 -0.436806

In [136]:
df.loc[('baz', 'two'):'foo']

A         B         C
baz two -0.085608  0.035008 -0.978340
foo one  0.624355  0.751345 -0.519235
    two  0.387675  0.630915  1.287880

In [137]:
df.loc[[('bar', 'two'), ('qux', 'one')]]

,,A,B,C
bar,two,1.383091,-0.726800,-0.190741
qux,one,1.758450,0.880635,-0.436806


Por último, el método `xs` de DataFrame con su argumento `level` facilita la selección de datos a un nivel particular de un MultiIndex.

In [138]:
df.xs('one', level=1)

,A,B,C
bar,0.689453,-1.036817,-0.299809
baz,-0.078887,0.328318,0.892238
foo,0.624355,0.751345,-0.519235
qux,1.758450,0.880635,-0.436806


<a id="aritmetica"></a>
### Aritmetica y Alineación de datos

Una característica importante de pandas para algunas aplicaciones es el comportamiento de la aritmética entre objetos con índices diferentes. Cuando se agregan objetos, si algún par de índices no es el mismo, el índice respectivo en el resultado será la unión de los pares de índices. Para los usuarios con experiencia en bases de datos, esto es similar a una combinación externa automática en las etiquetas de índice. 

In [139]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [140]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [141]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

La alineación de datos internos introduce valores `NA` en las ubicaciones de las etiquetas que no se superponen. Los valores `NA` se propagarán en otros cálculos aritméticos.

En el caso de un DataFrame, la alineación se realiza tanto en las filas como en las columnas:

In [142]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [143]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [144]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


Si se agregan objetos DataFrame sin etiquetas de fila o columna en común, el resultado contendrá todos valores nulos:

In [145]:
df1 = pd.DataFrame({'A': [1, 2]})
df1

,A
0,1
1,2


In [146]:
df2 = pd.DataFrame({'B': [3, 4]})
df2

,B
0,3
1,4


In [147]:
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


#### Métodos aritméticos con rellenado de valores

En las operaciones aritméticas entre objetos indexados de forma diferente, es posible que desee rellenar con un valor especial, como 0, cuando se encuentra una etiqueta de eje en un objeto pero no en el otro:

In [148]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [149]:
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
df2.loc[1, 'b'] = np.nan
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [150]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [151]:
df2.add(df1, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


La siguiente tabla muestra diferentes métodos aritméticos flexibles. Cada uno de ellos tiene una contraparte, comenzando con la letra r, que tiene argumentos invertidos.

|Método|Descripción|
|---|:---|
|add, radd|Métodos de suma (+)|
|sub, rsub|Métodos de resta (-)|
|div, rdiv|Métodos de división (/)|
|floordiv, rfloordiv|Métodos de floor división (//)|
|mul, rmul|Métodos de multiplicación (\*)|
|pow, rpow|Métodos de exponenciación (\**)|


De manera relacionada, al reindexar una serie o un marco de datos, también se puede especificar un valor de relleno diferente:

In [152]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


#### Operaciones entre DataFrame y Series

De igual forma que Numpy permitía operaciones entre arrays de diferentes dimensiones mediante la técnica de difusión (*broadcasting*), las operaciones entre DataFrames y Series son similares:

In [153]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [154]:
arr[0]

array([0., 1., 2., 3.])

In [155]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [156]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [157]:
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

De forma predeterminada, la aritmética entre el DataFrame y la serie coincide con el índice de la serie en las columnas del marco de datos, transmitiéndose por las filas:

In [158]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


Si no se encuentra un valor de índice en las columnas del marco de datos o en el índice de la serie, los objetos se volverán a indexar para formar la unión:

In [159]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


Si se desea difundir sobre las columnas, haciendo coincidir las filas, se debe utilizar uno de los métodos aritméticos. Por ejemplo:

In [160]:
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [161]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [162]:
frame.sub(series3, axis='index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


El número de eje que se pasa es el eje con el que se desea coincidir. En este caso, queremos hacer coincidir en el índice de la fila del DataFrame (axis = 'index' o axis = 0) y transmitir a través del mismo.

<a id="mapeo"></a>
### Aplicación y mapeo de funciones

Las funciones universales Numpy (ufuncs) también funcionan sobre objetos pandas.

In [163]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-1.971400,0.586063,1.323001
Ohio,0.407445,-1.239632,0.258292
Texas,0.641365,0.601035,-0.170559
Oregon,-0.800452,0.905325,-1.925462


In [164]:
np.abs(frame)

,b,d,e
Utah,1.971400,0.586063,1.323001
Ohio,0.407445,1.239632,0.258292
Texas,0.641365,0.601035,0.170559
Oregon,0.800452,0.905325,1.925462


Otra operación frecuente es aplicar una función en matrices unidimensionales a cada columna o fila. El método `apply` de DataFrame hace exactamente esto:

In [165]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    2.612765
d    2.144957
e    3.248463
dtype: float64

Aquí, la función `f`, que calcula la diferencia entre el máximo y el mínimo de una serie, se invoca una vez en cada columna en `frame`. El resultado es una Serie que tiene las columnas de `frame` como su índice.

Si pasa el `axis = 'columns'` a `apply`, la función se invocará una vez por fila:

In [166]:
frame.apply(f, axis='columns')

Utah      3.294401
Ohio      1.647077
Texas     0.811924
Oregon    2.830787
dtype: float64

Muchas de las estadísticas de matriz más comunes (como suma y media) son métodos DataFrame, por lo que no es necesario usar `apply`. La función pasada a `apply` no necesita devolver un valor escalar, también puede devolver una serie con múltiples valores:

In [167]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

frame.apply(f)

,b,d,e
min,-1.971400,-1.239632,-1.925462
max,0.641365,0.905325,1.323001


También se pueden usar funciones de Python aplicables a elementos. Supongamos que desea obtener una cadena con formato de cada valor de punto flotante en `frame`. Se puede hacer esto usando `applymap`:

In [168]:
format = lambda x: '%.2f' % x
frame.applymap(format)

C:\Users\mfcardenas\AppData\Local\Temp\ipykernel_38348\1073433956.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame.applymap(format)


,b,d,e
Utah,-1.97,0.59,1.32
Ohio,0.41,-1.24,0.26
Texas,0.64,0.60,-0.17
Oregon,-0.80,0.91,-1.93


La razón para el nombre `applymap` es que las Series tiene un método `map` para aplicar una función de elementos:

In [169]:
frame['e'].map(format)

Utah       1.32
Ohio       0.26
Texas     -0.17
Oregon    -1.93
Name: e, dtype: object

<a id="ordenacion"></a>
### Ordenación y clasificación

La clasificación de un conjunto de datos por algún criterio es otra operación incorporada importante. Para ordenar lexicográficamente por índice de fila o columna, se usa el método `sort_index`, que devuelve un nuevo objeto ordenado. 

In [170]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

Los DataFrame pueden ordenarse por el índice o por los ejes. Los datos son ordenados de forma ascendente por defecto.

In [171]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [172]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [173]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


Las series pueden ordenarse por sus valores haciendo uso del método `sort_values`:

In [174]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

Los valors `NA` (`NaN`) se ordenan por defecto al final de las series:

In [175]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

Al ordenar un DataFrame, se pueden usar los datos en una o más columnas como claves de clasificación. Para hacerlo, se pasa uno o más nombres de columna a la opción `by` de `sort_values`:

In [176]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [177]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [178]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


La clasificación asigna rangos de uno a la cantidad de puntos de datos válidos en una matriz. Los métodos de clasificación (`rank`) para Series y DataFrame calculan rangos de datos numéricos (`1 a N`) a lo largo del eje. A los valores iguales se les asigna un rango que es el promedio de los rangos de esos valores.

In [179]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

Los rangos también se pueden asignar según el orden en que se observan en los datos:

In [180]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [181]:
# Asigna valores de empate al rango máximo en el grupo
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

Los DataFrame pueden definir rangos sobre filas y sobre columnas:

In [182]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [183]:
frame.rank(axis='columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


La siguiente lista muestra los métodos para la generación de los rangos de clasificación:

|Métdodo|Descripción|
|---|:---|
|'average'|Predeterminado: asigna el rango promedio a cada entrada igual|
|'min'|Utiliza el rango mínimo para todo el grupo|
|'max'|Utiliza el rango máximo para todo el grupo|
|'first'|Asignar rangos en el orden en que aparecen los valores en los datos|
|'dense'|Como 'min', pero los rangos siempre aumentan en 1 entre los grupos en lugar del número de elementos iguales en un grupo|


<a id="etiquetas_duplicadas"></a>
### Índices sobre ejes con etiquetas duplicadas

Hasta ahora, todos los ejemplos que hemos visto tienen etiquetas de eje únicas (valores de índice). Si bien muchas funciones de pandas (como la reindexación) requieren que las etiquetas sean únicas, no es obligatorio. Consideremos una pequeña serie con índices duplicados:

In [184]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

La propiedad `is_unique` del índice puede decir si sus etiquetas son únicas o no:

In [185]:
obj.index.is_unique

False

La selección de datos es una de las principales cosas que se comporta de manera diferente con los duplicados. La indexación de una etiqueta con varias entradas devuelve una Serie, mientras que las entradas individuales devuelven un valor escalar:

In [186]:
obj['a']

a    0
a    1
dtype: int64

La misma lógica se extiende a la indexación de filas en un DataFrame:

In [187]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,-1.037181,-0.857624,0.338545
a,1.383681,0.393325,0.758027
b,0.549508,0.847885,-2.148597
b,0.176945,1.174874,-0.263617


In [188]:
df.loc['b']

,0,1,2
b,0.549508,0.847885,-2.148597
b,0.176945,1.174874,-0.263617


<a id="datos_duplicados"></a>
### Datos duplicados

Si desea identificar y eliminar filas duplicadas en un DataFrame, existen dos métodos que lo ayudarán: duplicado y drop_duplicates. Cada uno toma como argumento las columnas a usar para identificar filas duplicadas.

- `duplicates` devuelve un vector booleano cuya longitud es el número de filas y que indica si una fila está duplicada.
- `drop_duplicates` elimina las filas duplicadas.

De forma predeterminada, la primera fila observada de un conjunto duplicado se considera única, pero cada método tiene un parámetro de mantenimiento para especificar los objetivos que se mantendrán.

- `keep = 'first'` (predeterminado): marca / suelta duplicados excepto la primera vez que ocurre.
- `keep = 'last'`: marca / suelta duplicados excepto la última aparición.
- `keep = False`: marca / suelta todos los duplicados.

In [189]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
                    'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'],
                    'c': np.random.randn(7)})
df2

,a,b,c
0,one,x,0.308733
1,one,y,0.955201
2,two,x,0.306664
3,two,y,-2.992844
4,two,x,-2.951319
5,three,x,-0.166667
6,four,x,-0.063486


In [190]:
df2.duplicated('a')

0    False
1     True
2    False
3     True
4     True
5    False
6    False
dtype: bool

In [191]:
df2.duplicated(['a', 'b'])

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [192]:
df2.drop_duplicates('a', keep='last')

,a,b,c
1,one,y,0.955201
4,two,x,-2.951319
5,three,x,-0.166667
6,four,x,-0.063486


Para eliminar los duplicados por valor de índice, use `Index.duplicated` y luego realice el corte. El mismo conjunto de opciones está disponible para el parámetro `keep`.

In [193]:
df3 = pd.DataFrame({'a': np.arange(6),
                    'b': np.random.randn(6)},
                   index=['a', 'a', 'b', 'c', 'b', 'a'])
df3

,a,b
a,0,1.194338
a,1,-1.395357
b,2,0.727873
c,3,-2.177478
b,4,0.752387
a,5,-0.093803


In [194]:
df3.index.duplicated()

array([False,  True, False, False,  True,  True])

In [195]:
df3[~df3.index.duplicated()]

,a,b
a,0,1.194338
b,2,0.727873
c,3,-2.177478


In [196]:
df3[~df3.index.duplicated(keep='last')]

,a,b
c,3,-2.177478
b,4,0.752387
a,5,-0.093803


<a id="estadisticas"></a>
## Usando estadísticas descriptivas

Los objetos pandas están equipados con un conjunto de métodos matemáticos y estadísticos comunes. La mayoría de éstos pertenecen a la categoría de reducciones o estadísticas de resumen, es decir, métodos que extraen un solo valor (como la suma o la media) de una serie o de los valores de las filas o columnas de un DataFrame. En comparación con los métodos similares que se encuentran en las matrices NumPy, tienen un manejo integrado de los datos faltantes. 

In [197]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [198]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [199]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

Los valores `NaN` se excluyen a menos que la sección completa (fila o columna en este caso) sea `NaN`. Esto se puede desactivar con la opción `skipna`:

In [200]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

Algunos métodos, como `idxmin` e `idxmax`, devuelven estadísticas indirectas como el valor de índice donde se alcanzan los valores mínimo o máximo:

In [201]:
df.idxmax()

one    b
two    d
dtype: object

Otros métodos son acumulaciones:

In [202]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Otros tipos de métodos no son una reducción ni una acumulación, `describe` es uno de esos ejemplos, que produce múltiples estadísticas de resumen en una sola petición:

In [203]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


Sobre datos no numéricos, `describe` produce estadísticas de resumen alternativas:

In [204]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

La siguiente lista muestra las estadísticas de resumen y descriptivas más habituales:

|Métdodo|Desccripción|
|---|:---|
|count|Número de valores no `NaN`|
|describe|Conjunto de estadísticas de resumen para Series o cada columna DataFrame|
|min, max|Valores mínimos y máximos|
|argmin, argmax|Índices (enteros) en los que se obtuvo el valor mínimo o máximo, respectivamente|
|idxmin, idxmax|Índices (etiquetas) en los que se obtuvo el valor mínimo o máximo, respectivamente|
|quantile|cuantil de la muestra de 0 a 1|
|sum|Suma de valores|
|mean|Media de valores|
|median|Media aritmética (50% cuantil) de valores|
|mad|Desviación absoluta media del valor medio|
|prod|Producto de todos los valores|
|var|Muestra la varianza de los valores|
|std|Muestra la desviación estándar de los valores|
|skew|Muestra la asimetría (tercer momento) de los valores|
|kurt|Muestra de curtosis (cuarto momento) de valores|
|cumsum|Suma acumulativa de valores|
|cummin, cummax|Mínimo o máximo acumulado de valores, respectivamente|
|cumprod|Producto acumulado de valores|
|diff|Calcula la primera diferencia aritmética (útil para series de tiempo)|
|pct_change|Calcula los cambios porcentuales|


<a id="recuentos"></a>
### Valores únicos, recuentos de valor y membresía

Otra clase de métodos extraen información sobre los valores contenidos en una Serie unidimensional. Por ejemplo, `unique`, devuelve una matriz con los valores únicos en una serie. Los valores únicos no necesariamente se devuelven ordenados, pero podrían ordenarse si fuera necesario (`uniques.sort()`).

In [205]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [206]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

El método `value_counts` devuelve una serie que contiene frecuencias de valores:

In [207]:
obj.value_counts()

c    3
a    3
b    2
d    1
Name: count, dtype: int64

Las series están ordenadas por valor en orden descendente por defecto, `value_counts` también está disponible como un método de pandas de nivel superior que se puede usar con cualquier matriz o secuencia:

In [208]:
pd.value_counts(obj.values, sort=False)

C:\Users\mfcardenas\AppData\Local\Temp\ipykernel_38348\3084372382.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(obj.values, sort=False)


c    3
a    3
d    1
b    2
Name: count, dtype: int64

El método `isin` realiza una verificación de membresía del conjunto vectorizado y puede ser útil para filtrar un conjunto de datos a un subconjunto de valores en una Serie o columna en un DataFrame:

In [209]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [210]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [211]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

Relacionado con `isin` está el método `Index.get_indexer`, que proporciona una matriz de índices resultado de aplicar elementos comunes en las series:

In [212]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2], dtype=int64)

---